# Merging/Joining Example
## Copyright (C) Princeton Consultants, 2017-2018
### First import pandas library

In [1]:
import pandas as pd

### Read in 3 tables

In [2]:
peoplehoursperday = pd.read_csv("peoplehoursperday.csv", index_col=[0,1])
peoplehoursperday.index.names=['person_id','day']
peoplehoursperday.head()

,,hours
person_id,day,
ADK0000000000ICFPT,Mo,0
BEL0000000001JDGQU,Mo,8
CFM0000000002KEHRV,Mo,0
DGN0000000003LFISW,Mo,0
EHO0000000004MGJTX,Mo,0


In [3]:
sitehoursperday = pd.read_csv("sitehoursperday.csv", index_col=[0,1])
sitehoursperday.index.names = ['site_id', 'day']
sitehoursperday.head()

hours
site_id            day       
ADK0000000364ICFPT Mo     8.0
                   Tu     8.0
                   We     8.0
                   Th     8.0
                   Fr     4.0

In [4]:
prefs = pd.read_csv("prefs.csv", index_col=[0,1])
prefs.head()


,,pref
person_id,site_id,
MPW0000000272UORBF,ADK0000000364ICFPT,64
ORY0000000196WQTDH,ADK0000000364ICFPT,32
QTA0000000146YSVFJ,ADK0000000364ICFPT,97
TWD0000000123BVYIM,ADK0000000364ICFPT,57
ADK0000000000ICFPT,ADK0000000390ICFPT,35


### For debugging purposes, do the steps one at a time. We need to reset the index on each table because we are using part of the index of the `prefs` table, and part of the index of the `peoplehoursperday` table

In [5]:
(
pd.merge(prefs.reset_index(),peoplehoursperday.reset_index(), 
         on='person_id', how='left')
).head()

,person_id,site_id,pref,day,hours
0,MPW0000000272UORBF,ADK0000000364ICFPT,64,Mo,8
1,MPW0000000272UORBF,ADK0000000364ICFPT,64,Tu,8
2,MPW0000000272UORBF,ADK0000000364ICFPT,64,We,8
3,MPW0000000272UORBF,ADK0000000364ICFPT,64,Th,4
4,MPW0000000272UORBF,ADK0000000364ICFPT,64,Fr,4


### We want the hours to correspond to the people, so we rename that column.

In [6]:
(
pd.merge(prefs.reset_index(),peoplehoursperday.reset_index(), 
         on='person_id', how='left')
    .rename(columns={'hours' : 'person_hours'})
).head()

,person_id,site_id,pref,day,person_hours
0,MPW0000000272UORBF,ADK0000000364ICFPT,64,Mo,8
1,MPW0000000272UORBF,ADK0000000364ICFPT,64,Tu,8
2,MPW0000000272UORBF,ADK0000000364ICFPT,64,We,8
3,MPW0000000272UORBF,ADK0000000364ICFPT,64,Th,4
4,MPW0000000272UORBF,ADK0000000364ICFPT,64,Fr,4


### Now do the second merge where we merge with the site hours. We do an inner join, because we want only the pairs that have matching days

In [7]:
(
pd.merge(prefs.reset_index(),peoplehoursperday.reset_index(), 
         on='person_id', how='left')
    .rename(columns={'hours' : 'person_hours'})
    .merge(sitehoursperday.reset_index(),
           on=['site_id','day'], how='inner')
).head()

,person_id,site_id,pref,day,person_hours,hours
0,MPW0000000272UORBF,ADK0000000364ICFPT,64,Mo,8,8.0
1,ORY0000000196WQTDH,ADK0000000364ICFPT,32,Mo,0,8.0
2,QTA0000000146YSVFJ,ADK0000000364ICFPT,97,Mo,6,8.0
3,TWD0000000123BVYIM,ADK0000000364ICFPT,57,Mo,6,8.0
4,MPW0000000272UORBF,ADK0000000364ICFPT,64,Tu,8,8.0


### Rename the site hours colum and store the result in a DataFrame

In [8]:
prefswithdaily = (
pd.merge(prefs.reset_index(),peoplehoursperday.reset_index(), 
         on='person_id', how='left')
    .rename(columns={'hours' : 'person_hours'})
    .merge(sitehoursperday.reset_index(),
           on=['site_id','day'], how='inner')
    .rename(columns={'hours' : 'site_hours'})
)
prefswithdaily.head()

,person_id,site_id,pref,day,person_hours,site_hours
0,MPW0000000272UORBF,ADK0000000364ICFPT,64,Mo,8,8.0
1,ORY0000000196WQTDH,ADK0000000364ICFPT,32,Mo,0,8.0
2,QTA0000000146YSVFJ,ADK0000000364ICFPT,97,Mo,6,8.0
3,TWD0000000123BVYIM,ADK0000000364ICFPT,57,Mo,6,8.0
4,MPW0000000272UORBF,ADK0000000364ICFPT,64,Tu,8,8.0


In [9]:
prefswithdaily.tail()

,person_id,site_id,pref,day,person_hours,site_hours
112359,WZG0000000204EYBLP,ZCJ0000000883HBEOS,89,Mo,2,0.5
112360,UXE0000000228CWZJN,ZCJ0000000883HBEOS,95,We,5,0.5
112361,WZG0000000204EYBLP,ZCJ0000000883HBEOS,89,We,5,0.5
112362,UXE0000000228CWZJN,ZCJ0000000883HBEOS,95,Th,5,0.5
112363,WZG0000000204EYBLP,ZCJ0000000883HBEOS,89,Th,1,0.5


### The actual hours is going to be the minimum of the two values.  Show how pandas does that. Note the shortcut to add a column without using `assign`.

In [10]:
prefswithdaily['daily_hours'] = prefswithdaily[['person_hours','site_hours']].min(axis=1)
prefswithdaily = prefswithdaily.drop(columns=['person_hours','site_hours'])[prefswithdaily.daily_hours > 0]
prefswithdaily.head()

,person_id,site_id,pref,day,daily_hours
0,MPW0000000272UORBF,ADK0000000364ICFPT,64,Mo,8.0
2,QTA0000000146YSVFJ,ADK0000000364ICFPT,97,Mo,6.0
3,TWD0000000123BVYIM,ADK0000000364ICFPT,57,Mo,6.0
4,MPW0000000272UORBF,ADK0000000364ICFPT,64,Tu,8.0
6,QTA0000000146YSVFJ,ADK0000000364ICFPT,97,Tu,6.0
